In [23]:
import pandas as pd
import re

from selenium import webdriver
from time import sleep
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains  # 用于鼠标悬停
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
# 启动 Edge 浏览器
driver = webdriver.Edge()

# 打开租房网
url = 'https://zu.fang.com/'
driver.get(url)

#等待页面加载
sleep(5)

In [26]:
# 找到地区选择的按钮，点击天津
hover_element = driver.find_element(By.CSS_SELECTOR, "div.s4Box")

actions = ActionChains(driver)
actions.move_to_element(hover_element).perform()

tianjin_city = driver.find_element(By.XPATH, "//div[@class='city20141104nr' and @id='cityi010']/a[text()='天津']")
tianjin_city.click()

sleep(5)

In [27]:
# 找到“滨海新区”对应的链接
binhai = driver.find_element(By.LINK_TEXT, "滨海新区")
binhai.click()

sleep(5)

In [33]:
# 找到“中新生态城”对应的链接
zx_city = driver.find_element(By.LINK_TEXT, "中新生态城")
zx_city.click()

sleep(5)

In [34]:
from selenium.common.exceptions import NoSuchElementException

house_data = []  # 房源数据汇总

while True:
    # 1. 找到房源列表并解析当前页数据
    try:
        shop_list_container = driver.find_element(By.CSS_SELECTOR, "div.houseList") #该页的总列表
    except NoSuchElementException:
        print("未找到房源列表")
        break  # 若连主容器都无法找到，直接退出循环
    
    # 找到每一条房源的dl标签
    dl_elements = shop_list_container.find_elements(By.TAG_NAME, "dl")

    for dl in dl_elements:
        try:
            title = dl.find_element(By.XPATH, ".//a").text.strip()  #标题
            house_info = dl.find_element(By.XPATH, ".//p[@class='font15 mt12 bold']").text.strip() #具体信息
            # 根据实际页面结构调整 XPath
            price = dl.find_element(By.XPATH, ".//span[@class='price']").text.strip()#价格
            house_data.append({
                "标题": title,
                "户型_面积等": house_info,
                "价格": price,
            })
        except Exception as e:
            print(f"跳过一条记录，错误：{e}")
            continue

    #  2. 找到“下一页”并点击翻页，直到找不到结束 
    try:
        sleep(2)
        # “下一页”按钮
        next_page = driver.find_element(By.LINK_TEXT, "下一页")
        next_page.click()
        sleep(3)
    except NoSuchElementException:
        print("没有找到下一页按钮, 爬取结束!")
        break

# 全部页面爬取结束后，将 house_data 转为 DataFrame
df = pd.DataFrame(house_data)
df

没有找到下一页按钮, 爬取结束!


,标题,户型_面积等,价格
0,,整租|2室2厅|89㎡|朝南,3000
1,,整租|2室1厅|83㎡|朝南,2800
2,,整租|3室2厅|135㎡|朝南北,4000
3,,整租|2室2厅|94㎡|朝南北,3200
4,,整租|2室2厅|80㎡|朝南北,2800
...,...,...,...
434,,整租|3室2厅|125㎡|朝南北,2800
435,,整租|2室2厅|90㎡|朝南,2200
436,,整租|3室2厅|126㎡|朝南,2800
437,,整租|3室2厅|102㎡|朝南北,2100


In [35]:
import pandas as pd
import re

# 假设 df 是之前爬取到的数据
df_cleaned = df.copy()

# 1. 提取平方数（㎡）：从 "户型_面积等" 中匹配第一个 "|" 和第二个 "|" 之间的内容
def extract_square_meters(house_info):
    try:
        # 此处匹配 | 后面空格、数字（包括小数点）和"㎡"的模式
        match = re.search(r"\|\s*([\d.]+)㎡\s*\|", house_info)
        return float(match.group(1)) if match else None
    except Exception as e:
        return None

df_cleaned["平方数（㎡）"] = df_cleaned["户型_面积等"].apply(extract_square_meters)


# 2. 处理价格（元/㎡），去掉 "元/㎡" 单位
df_cleaned["价格（元/㎡）"] = df_cleaned["价格"].str.replace("元/㎡", "", regex=False).astype(float)

# 3. 仅保留需要的两列
df_cleaned = df_cleaned[["平方数（㎡）", "价格（元/㎡）"]]

# 将清洗后的 DataFrame 存储到 CSV 文件，不保存索引
df_cleaned.to_csv("zhongxin_rental.csv", index=False)

print("数据已保存")

数据已保存
